In [1]:
URL ="https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max=2022-11-24&date_received_min=2022-11-23&field=all&format=json"

In [5]:
import json
import requests

In [2]:

data = requests.get(URL)


In [22]:
with open("../filtered_data.json", "w") as file_obj:
    finance_complaint_data = list(map(lambda x: x["_source"],
                                                json.loads(data.content)))
                                    

    json.dump(finance_complaint_data, file_obj)


In [23]:
import pandas as pd

df=pd.read_json("../filtered_data.json")

df.head()

,product,complaint_what_happened,date_sent_to_company,issue,sub_product,zip_code,tags,complaint_id,timely,consumer_consent_provided,company_response,submitted_via,company,date_received,state,consumer_disputed,company_public_response,sub_issue
0,"Credit reporting, credit repair services, or o...",,2022-11-23T12:00:00-05:00,Improper use of your report,Credit reporting,33880.0,None,6232086,Yes,None,In progress,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2022-11-23T12:00:00-05:00,FL,N/A,None,Credit inquiries on your report that you don't...
1,"Credit reporting, credit repair services, or o...",,2022-11-23T12:00:00-05:00,Incorrect information on your report,Credit reporting,89052.0,None,6231821,Yes,None,In progress,Web,"EQUIFAX, INC.",2022-11-23T12:00:00-05:00,NV,N/A,None,Information belongs to someone else
2,"Credit reporting, credit repair services, or o...",,2022-11-23T12:00:00-05:00,Problem with a credit reporting company's inve...,Credit reporting,7062.0,None,6231771,Yes,None,In progress,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2022-11-23T12:00:00-05:00,NJ,N/A,None,Their investigation did not fix an error on yo...
3,"Credit reporting, credit repair services, or o...",,2022-11-23T12:00:00-05:00,Improper use of your report,Credit reporting,90008.0,None,6232192,Yes,None,In progress,Web,"EQUIFAX, INC.",2022-11-23T12:00:00-05:00,CA,N/A,None,Reporting company used your report improperly
4,"Credit reporting, credit repair services, or o...",,2022-11-23T12:00:00-05:00,Problem with a credit reporting company's inve...,Credit reporting,92395.0,None,6232276,Yes,None,In progress,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2022-11-23T12:00:00-05:00,CA,N/A,None,Their investigation did not fix an error on yo...


In [24]:
df.shape

(1610, 18)

In [25]:

from pyspark.sql import SparkSession

In [2]:
# spark.sparkContext.stop()

In [26]:
spark = SparkSession.builder.master('local[*]').appName('finance_complaint')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/27 01:08:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
spark.sparkContext._conf.getAll()

[('spark.driver.host', '4d680389b3c9'),
 ('spark.sql.warehouse.dir',
  'file:/config/workspace/Notebook/spark-warehouse'),
 ('spark.app.startTime', '1669491486652'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '6g'),
 ('spark.app.name', 'finance_complaint'),
 ('spark.executor.memoryOverhead', '8g'),
 ('spark.driver.memory', '6g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.instances', '1'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '44057'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1669491488284'),
 ('spark.ui.showConsoleProgress', 'true')]

In [28]:
df = spark.read.json("../filtered_data.json")

In [29]:
df.count()

1610

In [30]:
df.show()

+--------------------+-----------------------+----------------+------------+-----------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+----------------+-------------+----+------+--------+
|             company|company_public_response|company_response|complaint_id|complaint_what_happened|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|     sub_product|submitted_via|tags|timely|zip_code|
+--------------------+-----------------------+----------------+------------+-----------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+----------------+-------------+----+------+--------+
|TRANSUNION INTERM...|                   null|     In progress|     6232086|

In [31]:
df.write.mode('append').parquet("../complaint_data")

In [33]:
df=spark.read.parquet("../complaint_data")

In [34]:
df.count()

1610

In [35]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- complaint_id: string (nullable = true)
 |-- complaint_what_happened: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- sub_product: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [37]:
df.select("consumer_disputed").distinct().show()

+-----------------+
|consumer_disputed|
+-----------------+
|              N/A|
+-----------------+



In [12]:
import pandas as pd

In [18]:
pd.date_range(start='2022-06-01',end='2022-11-11',freq='M').astype('str').tolist()

['2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31']